In [13]:
import numpy as np
import pandas as pd

In [14]:
# Laading the dataset
data = pd.read_excel('Employee_Salary.xlsx')             # NaN , None, NA will automatically converts into np.nan
data

,Name,Age,Height (cm),Weight (kg),Income ($)
0,?,32,178,75,50000
1,Sarah Wong,28,163,60,65000
2,Michael Lee,Nan,NaN,?,-9999999
3,Emma Davis,35,170,65,55000
4,Null,40,175,80,75000
5,Jennifer Kim,27,-,55,60000
6,James Rodriguez,31,182,NaN,70000
7,NaN,29,n/A,62,none
8,Daniel Harris,31,180,85,72000
9,Olivia Johnson,33,-9999999,70,67000


In [15]:
# convert the characters in dataframe to upper case
data = data.applymap(lambda s: s.upper() if type(s) == str else s)

# general represntation of missing value
gen_miss_val_rep = ['N/A', 'NA', 'NULL', '?', '', '-', 'NONE','$']   # big numbers like  +/- 999999 , inbuilt: np.nan, None

for i in gen_miss_val_rep:
    data.replace(i,np.nan,inplace=True)    
    
# columns with missing value
empty_columns = []

for i in range(data.shape[1]):
    if data.describe(include = 'all').iloc[0,i] != data.shape[0]:
        empty_columns.append(data.describe(include = 'all').columns[i])

print('Columns with missing values: ',empty_columns)

Columns with missing values:  ['Name', 'Age', 'Height (cm)', 'Weight (kg)', 'Income ($)']


In [16]:
# all the non - numerical missing value represntation has been replaced with NaN
data

,Name,Age,Height (cm),Weight (kg),Income ($)
0,NaN,32,178.0,75.0,50000.0
1,SARAH WONG,28,163.0,60.0,65000.0
2,MICHAEL LEE,NAN,NaN,NaN,-9999999.0
3,EMMA DAVIS,35,170.0,65.0,55000.0
4,NaN,40,175.0,80.0,75000.0
5,JENNIFER KIM,27,NaN,55.0,60000.0
6,JAMES RODRIGUEZ,31,182.0,NaN,70000.0
7,NaN,29,NaN,62.0,NaN
8,DANIEL HARRIS,31,180.0,85.0,72000.0
9,OLIVIA JOHNSON,33,-9999999.0,70.0,67000.0


In [17]:
# Removing Outliers

# list of dataframe containing column wise removed outlier 
outlier_removed =[]

int_var = list(data.describe().columns)

for i in int_var:
    
# Calculate the 25th and 75th percentiles
    Q1 = np.nanpercentile(data.dropna(subset = i)[i], 25)
    Q3 = np.nanpercentile(data.dropna(subset = i)[i], 75)

# Calculate the IQR
    IQR = Q3 - Q1

# Define the lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

# Filter out rows where the values are outside the bounds
    outlier_removed_df = data[data[i].isnull() | ((data[i] >= lower_bound) & (data[i] <= upper_bound))]  # will remove Nan as well
    outlier_removed.append((i,outlier_removed_df))

In [18]:
outlier_column = input('Enter the coumns from which you want to remove outliers (Column name seperated by,):')

Enter the coumns from which you want to remove outliers (Column name seperated by,):Height (cm),Income ($)


In [19]:
# Transforimg the input into list contaning pin_fileds

outlier_column = outlier_column.split(',')

for i in range(len(outlier_column)):
    outlier_column[i] = outlier_column[i].strip()
    
outlier_column

['Height (cm)', 'Income ($)']

In [20]:
index_set = set(outlier_removed[int_var.index(outlier_column[0])][1].index)

for i in outlier_column[1:]:
    index_set = index_set.intersection(set(outlier_removed[int_var.index(i)][1].index))

data = data.loc[list(index_set)]

# outlies have been removed from Height(cm) and Income($) Column
data

,Name,Age,Height (cm),Weight (kg),Income ($)
0,NaN,32,178.0,75.0,50000.0
1,SARAH WONG,28,163.0,60.0,65000.0
3,EMMA DAVIS,35,170.0,65.0,55000.0
4,NaN,40,175.0,80.0,75000.0
5,JENNIFER KIM,27,NaN,55.0,60000.0
6,JAMES RODRIGUEZ,31,182.0,NaN,70000.0
7,NaN,29,NaN,62.0,NaN
8,DANIEL HARRIS,31,180.0,85.0,72000.0
10,PAUL HOLLAND,NaN,169.0,54.0,NaN
11,JACOB ROSS,37,170.0,69.0,57000.0


In [21]:
imp_fields =input('Enter the columns for whch you want to handle missing value (columns should be sepearted by ,): ')

Enter the columns for whch you want to handle missing value (columns should be sepearted by ,): Income ($)


In [22]:
# Transforimg the input into list contaning pin_fileds

imp_fields = imp_fields.split(',')

for i in range(len(imp_fields)):
    imp_fields[i] = imp_fields[i].strip()
    
imp_fields

['Income ($)']

In [23]:
# Either drop na or replace it with something meaning full

# data = data.dropna(subset = imp_fields)

# Repcing missing value with a mean
for i in imp_fields:
    data.loc[:,i]=data.loc[:,i].replace(np.nan,data[i].mean().round(1))

# Missing value in income has been replaced by mean income value
data

,Name,Age,Height (cm),Weight (kg),Income ($)
0,NaN,32,178.0,75.0,50000.0
1,SARAH WONG,28,163.0,60.0,65000.0
3,EMMA DAVIS,35,170.0,65.0,55000.0
4,NaN,40,175.0,80.0,75000.0
5,JENNIFER KIM,27,NaN,55.0,60000.0
6,JAMES RODRIGUEZ,31,182.0,NaN,70000.0
7,NaN,29,NaN,62.0,63000.0
8,DANIEL HARRIS,31,180.0,85.0,72000.0
10,PAUL HOLLAND,NaN,169.0,54.0,63000.0
11,JACOB ROSS,37,170.0,69.0,57000.0
